In [1]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly' #'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [2]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [3]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [4]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, l.state_pm, t.total_flow ,
t.avg_speed
from traffic_bay t, pemslocs l
where t.timestamp between '2021-01-22 15:00:00' and '2021-01-22 15:03:00'
and l.sid = t.station and l.stype = 'ML'
"""
snap=pd.read_sql(query, engine, index_col="sid")
snap.head()

,longitude,latitude,stype,fwy,direc,abs_pm,state_pm,total_flow,avg_speed
sid,,,,,,,,,
400001,-121.901149,37.364085,ML,101,N,387.897,38.26,254,63.5
400017,-121.945440,37.253303,ML,85,S,10.070,10.07,403,57.4
400030,-121.906538,37.359087,ML,880,S,3.640,3.64,346,58.4
400040,-121.873109,37.294949,ML,87,S,2.860,2.87,330,56.0
400045,-121.902233,37.363402,ML,880,N,4.020,4.02,397,52.2


In [5]:
pd.read_sql("select * from pemslocs limit 1", engine)
#print(len(snap))

,sid,fwy,direc,district,county,city,state_pm,abs_pm,latitude,longitude,length,stype,lanes,name
0,308511,50,E,3,17,None,31.627,60.162,38.761062,-120.569835,5.0,ML,2,Sly Park Rd


In [6]:
snap["total_flow"]=snap["total_flow"].astype(float)

In [7]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['101N', '85S', '880S', '87S', '880N', '85N', '101S', '17S', '280S',
       '237W', '237E', '87N', '17N', '280N', '680S', '680N'], dtype=object)

In [8]:
fwdir='101S' #'94W'
sel=snap.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
sel.head()

,longitude,latitude,stype,fwy,direc,abs_pm,state_pm,total_flow,avg_speed,fwdir
sid,,,,,,,,,,
402360,-121.842361,37.329591,ML,101,S,383.669,34,503.0,58.5,101S
401906,-121.848031,37.335259,ML,101,S,384.169,34.5,446.0,54.6,101S
401891,-121.855795,37.343293,ML,101,S,384.869,35.2,612.0,52.2,101S
400160,-121.860589,37.348392,ML,101,S,385.309,35.64,444.0,63.6,101S
400440,-121.865827,37.353727,ML,101,S,385.779,36.11,275.0,62.8,101S


In [9]:
#function to move fwys in separate directions apart for visualization
def spread_lanes(sdf, fwdir, latspread=0.0005, lonspread=0.0005):
    direc=fwdir[-1]
    
    # this assumes right-hand traffic
    if direc == 'W':
         sdf["latitude"] += latspread
    elif direc == "E":
        sdf["latitude"] -= latspread
    elif direc == "S":
        sdf["longitude"] -= lonspread
    elif direc == "N":
        sdf["longitude"] += lonspread
    
    return sdf
    
#selhr=pd.read_sql("select * from usdot u where u.fwdir = '94W' order by u.abs_pm", engine)
#display(selhr.head(n=1))
#spread_lanes(selhr, '94W').head(n=1)

In [10]:
#interp1d?
import numpy as np
np.argmin(sel["abs_pm"])

0

In [11]:
#interp1d?

In [12]:
hrdfs=[]

from scipy.interpolate import interp1d

for fwdir in snap["fwdir"].unique():
    print("\rInterpolating average speed along %s" % fwdir, end="")
    sel=snap.query('fwdir == "%s"' % fwdir).sort_values(by="abs_pm")
    selhr=pd.read_sql("select * from usdot u where u.fwdir = '%s' order by u.abs_pm" % fwdir, engine)
    selhr=spread_lanes(selhr, fwdir)

    if len(sel) > 1:
        #fillvalmin=sel.loc[minidx,"abs_pm"]
        #fillvalmax=sel.loc[maxidx,"abs_pm"]
        interpmod = interp1d(sel["abs_pm"].values, sel["avg_speed"].values, bounds_error=False)
                            #fill_value='extrapolate')#[fillvalmin, villvalmax])
        selhr["avg_speed"] = interpmod(selhr["abs_pm"].values)
        #selhr.dropna(axis="index", how="any", subset=["avg_speed"], inplace=True)
    #selhr["avg_speed"].describe()
        hrdfs.append(selhr)
print(" done.")

selhr=pd.concat(hrdfs)
selhr.reset_index(drop=True, inplace=True)

Interpolating average speed along 680N done.


In [13]:
def speed2cat(speed):
    if speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    elif np.isnan(speed):
        return 'no data'
    else:
        return 'very slow'
        
selhr["speed_cat"]=selhr["avg_speed"].apply(speed2cat)

In [14]:
selhr.head()

,usdid,fwdir,latitude,longitude,abs_pm,avg_speed,speed_cat
0,1,101N,36.917391,-121.547509,349.300000,NaN,no data
1,2,101N,36.917411,-121.547501,349.301451,NaN,no data
2,3,101N,36.917441,-121.547491,349.303596,NaN,no data
3,4,101N,36.917471,-121.547481,349.305741,NaN,no data
4,5,101N,36.917491,-121.547474,349.307176,NaN,no data


In [15]:
# %load road_group_split.py
def road_group_split(indf, splitcat="speed_cat", 
                     keep_cols=["latitude", "longitude", "abs_pm", "avg_speed","fwdir"]):
    
    odict={}
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,"fwdir"] != indf.loc[i0,"fwdir"]:
            p+=1
            
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat] and indf.loc[n,"fwdir"] == indf.loc[i0,"fwdir"]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selhrg=road_group_split(selhr)
selhrg.tail()

,latitude,longitude,abs_pm,avg_speed,fwdir,speed_cat,linegroup
8061,37.463662,-121.902761,9.858517,NaN,680N,no data,171
8062,37.463810,-121.902878,9.870589,NaN,680N,no data,171
8063,37.463950,-121.902988,9.881989,NaN,680N,no data,171
8064,37.464358,-121.903274,9.914249,NaN,680N,no data,171
8065,37.464540,-121.903417,9.929069,NaN,680N,no data,171


In [16]:
cdm={"fast": "green",
     "moderate": "orange",
     "slow": "red",
     "very slow": "brown",
     "no data": "gray"}

In [17]:
lonmax=-117.09
lonmin=-120.
latmax=32.82
latmin=32.65

def prune_roadsdf(indf, lonmin, lonmax, latmin, latmax):
    c1=indf["latitude"] > latmin
    c2=indf["latitude"] < latmax
    c3=indf["longitude"] > lonmin
    c4=indf["longitude"] < lonmax
    idx = c1 & c2 & c3 & c4
    return indf.loc[idx,:]

selhrgp=prune_roadsdf(selhrg, lonmin, lonmax, latmin, latmax)

In [18]:
#selhrg.reset_index(drop=True, inplace=True)
#selhrg

In [21]:
#del fig
fig = px.line_mapbox(
#fig=px.scatter_mapbox
    selhrg, lat="latitude", lon="longitude", color="speed_cat",
                        #mapbox_style="stamen-terrain",
                        mapbox_style="carto-positron",
                        #color_continuous_scale="aggrnyl" ,
                        line_group = "linegroup",
                        hover_data=["abs_pm", "avg_speed"],
                        #center = {"lat": 32.77, "lon": -117.16},
                        color_discrete_map= cdm,
                        #zoom=12,
                        height=800, width=800)

for dat in fig.data:
    dat["line"]["width"] = 3

snap["size"] = 5
fig2 = px.scatter_mapbox(snap, lat="latitude", lon="longitude",
                        hover_data=["stype","fwy","direc","abs_pm","total_flow","avg_speed"],
                        mapbox_style="stamen-terrain",
                         color="stype",
                        color_discrete_map={"ML": "blue"},
                        size="size",
                        opacity=0.35,
                        size_max=5)
#for dat in fig2.data:
fig.add_trace(fig2.data[0])
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [20]:
fig.write_image("traffic_layer_bay_w_usdot.jpg")